In [1]:
import graphlab as gl

In [2]:
filename = "./price_EUR_USD_*H1.csv"
sf_eur_usd_h1 = gl.SFrame.read_csv(filename)

[INFO] This non-commercial license of GraphLab Create is assigned to yuecong1104@gmail.comand will expire on August 13, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-4184 - Server binary: /Library/Python/2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1439771798.log
[INFO] GraphLab Server Version: 1.5.2


PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2002-01-01T00%3A00%3A00Z_H1.csv
PROGRESS: Read 100 lines. Lines per second: 1289.37
PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2002-07-01T00%3A00%3A00Z_H1.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.078078 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int,int,float,float,float,float,float,float,float,float,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2002-01-01T00%3A00%3A00Z_H1.csv
PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2002-07-01T00%3A00%3A00Z_H1.csv
PROGRESS: Finished parsing file /Users/yuecong/autotrade/price_EUR_USD_2003

In [3]:
#sf_eur_usd_h1.show()

In [4]:
sf_eur_usd_h1.add_column( sf_eur_usd_h1['year'] .astype(str)+ '-' + sf_eur_usd_h1['month'].astype(str) + 
                         '-' + sf_eur_usd_h1['day'].astype(str) , 'date' )
sf_eur_usd_h1.add_column((sf_eur_usd_h1['openBid'] + sf_eur_usd_h1['openAsk']) /2 , 'open')
sf_eur_usd_h1.add_column((sf_eur_usd_h1['highBid'] + sf_eur_usd_h1['highAsk']) /2 , 'high')
sf_eur_usd_h1.add_column((sf_eur_usd_h1['lowBid'] + sf_eur_usd_h1['lowAsk']) /2 , 'low')
sf_eur_usd_h1.add_column((sf_eur_usd_h1['closeBid'] + sf_eur_usd_h1['closeAsk']) /2 , 'close')
sf_eur_usd_h1.remove_columns(['openBid','openAsk','highBid','highAsk','lowBid','lowAsk','closeBid','closeAsk','complete'])

year,month,day,hour,minute,second,volume,date,open,high,low,close
2002,5,6,20,0,0,1,2002-5-6,0.91825,0.91825,0.91825,0.91825
2002,5,7,20,0,0,1,2002-5-7,0.91535,0.91535,0.91535,0.91535
2002,5,8,20,0,0,1,2002-5-8,0.90435,0.90435,0.90435,0.90435
2002,5,9,20,0,0,1,2002-5-9,0.90935,0.90935,0.90935,0.90935
2002,5,10,20,0,0,1,2002-5-10,0.914,0.914,0.914,0.914
2002,5,10,22,0,0,1,2002-5-10,0.9141,0.9141,0.9141,0.9141
2002,5,13,20,0,0,1,2002-5-13,0.91155,0.91155,0.91155,0.91155
2002,5,14,20,0,0,1,2002-5-14,0.90275,0.90275,0.90275,0.90275
2002,5,15,20,0,0,1,2002-5-15,0.91255,0.91255,0.91255,0.91255
2002,5,16,20,0,0,1,2002-5-16,0.91195,0.91195,0.91195,0.91195


In [5]:
sf_eur_usd_h1 = sf_eur_usd_h1.sort(['year','month','day','hour','minute','second'])

In [6]:
sf_eur_usd_h1_tmp = sf_eur_usd_h1

In [7]:
sf_eur_usd_h1_tmp.add_column(sf_eur_usd_h1_tmp['open'] *sf_eur_usd_h1_tmp['volume'] ,'open_volume')
sf_eur_usd_h1_tmp.add_column(sf_eur_usd_h1_tmp['high'] *sf_eur_usd_h1_tmp['volume'] ,'high_volume')
sf_eur_usd_h1_tmp.add_column(sf_eur_usd_h1_tmp['low'] *sf_eur_usd_h1_tmp['volume'] ,'low_volume')
sf_eur_usd_h1_tmp.add_column(sf_eur_usd_h1_tmp['close'] *sf_eur_usd_h1_tmp['volume'] ,'close_volume')

year,month,day,hour,minute,second,volume,date,open,high,low,close,open_volume
2002,5,6,20,0,0,1,2002-5-6,0.91825,0.91825,0.91825,0.91825,0.91825
2002,5,7,20,0,0,1,2002-5-7,0.91535,0.91535,0.91535,0.91535,0.91535
2002,5,8,20,0,0,1,2002-5-8,0.90435,0.90435,0.90435,0.90435,0.90435
2002,5,9,20,0,0,1,2002-5-9,0.90935,0.90935,0.90935,0.90935,0.90935
2002,5,10,20,0,0,1,2002-5-10,0.914,0.914,0.914,0.914,0.914
2002,5,10,22,0,0,1,2002-5-10,0.9141,0.9141,0.9141,0.9141,0.9141
2002,5,13,20,0,0,1,2002-5-13,0.91155,0.91155,0.91155,0.91155,0.91155
2002,5,14,20,0,0,1,2002-5-14,0.90275,0.90275,0.90275,0.90275,0.90275
2002,5,15,20,0,0,1,2002-5-15,0.91255,0.91255,0.91255,0.91255,0.91255
2002,5,16,20,0,0,1,2002-5-16,0.91195,0.91195,0.91195,0.91195,0.91195


In [8]:
sf_eur_usd_h1_tmp.show()

Canvas is accessible via web browser at the URL: http://localhost:60611/index.html
Opening Canvas in default web browser.


In [9]:
sf_eur_usd_h1_tmp[sf_eur_usd_h1_tmp['volume'] >100].shape

(66493, 16)

In [10]:
sf_eur_usd_h1_tmp.shape

(69296, 16)

In [11]:
sf_eur_usd_d1_tmp = sf_eur_usd_h1_tmp.groupby(['day','month','year','date'],
                                              [gl.aggregate.SUM('open_volume'),
                                               gl.aggregate.SUM('high_volume'),
                                               gl.aggregate.SUM('low_volume'),
                                               gl.aggregate.SUM('close_volume'),
                                               gl.aggregate.SUM('volume'),
                                              ]
                                             )

In [12]:
sf_eur_usd_d1_tmp = sf_eur_usd_d1_tmp.sort(['year','month','day'])

In [13]:
sf_eur_usd_d1_tmp

date,day,month,year,Sum of open_volume,Sum of high_volume,Sum of low_volume,Sum of close_volume,Sum of volume
2002-5-6,6,5,2002,0.91825,0.91825,0.91825,0.91825,1
2002-5-7,7,5,2002,0.91535,0.91535,0.91535,0.91535,1
2002-5-8,8,5,2002,0.90435,0.90435,0.90435,0.90435,1
2002-5-9,9,5,2002,0.90935,0.90935,0.90935,0.90935,1
2002-5-10,10,5,2002,1.8281,1.8281,1.8281,1.8281,2
2002-5-13,13,5,2002,0.91155,0.91155,0.91155,0.91155,1
2002-5-14,14,5,2002,0.90275,0.90275,0.90275,0.90275,1
2002-5-15,15,5,2002,0.91255,0.91255,0.91255,0.91255,1
2002-5-16,16,5,2002,0.91195,0.91195,0.91195,0.91195,1
2002-5-17,17,5,2002,0.9209,0.9209,0.9209,0.9209,1


In [14]:
sf_eur_usd_d1_tmp.add_column(sf_eur_usd_d1_tmp['Sum of open_volume'] /sf_eur_usd_d1_tmp['Sum of volume'] ,'open')
sf_eur_usd_d1_tmp.add_column(sf_eur_usd_d1_tmp['Sum of high_volume'] /sf_eur_usd_d1_tmp['Sum of volume'] ,'high')
sf_eur_usd_d1_tmp.add_column(sf_eur_usd_d1_tmp['Sum of low_volume'] /sf_eur_usd_d1_tmp['Sum of volume'] ,'low')
sf_eur_usd_d1_tmp.add_column(sf_eur_usd_d1_tmp['Sum of close_volume'] /sf_eur_usd_d1_tmp['Sum of volume'] ,'close')
sf_eur_usd_d1_tmp.rename({'Sum of volume': 'volume'})
sf_eur_usd_d1_tmp.remove_columns(['Sum of open_volume','Sum of high_volume','Sum of low_volume','Sum of close_volume'])

date,day,month,year,volume,open,high,low,close
2002-5-6,6,5,2002,1,0.91825,0.91825,0.91825,0.91825
2002-5-7,7,5,2002,1,0.91535,0.91535,0.91535,0.91535
2002-5-8,8,5,2002,1,0.90435,0.90435,0.90435,0.90435
2002-5-9,9,5,2002,1,0.90935,0.90935,0.90935,0.90935
2002-5-10,10,5,2002,2,0.91405,0.91405,0.91405,0.91405
2002-5-13,13,5,2002,1,0.91155,0.91155,0.91155,0.91155
2002-5-14,14,5,2002,1,0.90275,0.90275,0.90275,0.90275
2002-5-15,15,5,2002,1,0.91255,0.91255,0.91255,0.91255
2002-5-16,16,5,2002,1,0.91195,0.91195,0.91195,0.91195
2002-5-17,17,5,2002,1,0.9209,0.9209,0.9209,0.9209


In [15]:
sf_eur_usd_d1 = sf_eur_usd_d1_tmp

# Use D1 to calculate features, h1 also could be used to calculate features with same logic
sf_eur_usd_features = sf_eur_usd_d1 

In [34]:
pip_unit = 10000
close_list = sf_eur_usd_features ['close']
close_minute_1 = gl.SArray([close_list[0]]).append(close_list[:-1])


In [37]:
close_minute_1

dtype: float
Rows: 4226
[0.91825, 0.91825, 0.91535, 0.90435, 0.90935, 0.9140499999999999, 0.91155, 0.9027499999999999, 0.91255, 0.91195, 0.9209, 0.9200999999999999, 0.9209499999999999, 0.9198500000000001, 0.92575, 0.92145, 0.92105, 0.9209, 0.92875, 0.93545, 0.9370499999999999, 0.9339, 0.9329000000000001, 0.9410499999999999, 0.93935, 0.93935, 0.94765, 0.944, 0.9432, 0.94455, 0.94825, 0.9439500000000001, 0.94345, 0.9456, 0.9453, 0.94445, 0.95185, 0.95745, 0.965, 0.97085, 0.97045, 0.9793499999999999, 0.9806, 0.98865, 0.9919, 0.9915, 0.9911, 0.9863500000000001, 0.98065, 0.9794499999999999, 0.9731000000000001, 0.9745999999999999, 0.99075, 0.9935499999999999, 0.98855, 0.98915, 0.9915, 0.9912000000000001, 1.0035500000000002, 1.0119500000000001, 1.00705, 1.0112, 1.0112, 1.0112, 1.0086499999999998, 0.98885, 0.99515, 1.0044499999999998, 0.9871000000000001, 0.9866999999999999, 0.9808, 0.9834, 0.9775, 0.98445, 0.9865999999999999, 0.9863999999999999, 0.98015, 0.96635, 0.97485, 0.9672000000000001, 0